In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import os 
import cv2
import random
import sys 
import csv
import pandas as pd
import bz2
from bz2 import BZ2File
from datetime import datetime
import shutil
from glob import glob
import matplotlib.pyplot as plt


from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

In [16]:
def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

def select_samples(arr, indices):
    ret = []
    for i in indices:
        ret.append(arr[i])
    return ret

def convert_to_datetime(time_str):
    if len(time_str) != 10:
        time_str = '0'*(10-len(time_str))+time_str
    date_time = datetime.strptime(time_str, '%y%m%d%H%M%S')
    return date_time

def nearst_match(target, candidate_mds):
    curr_diff = float('inf')
    for curr in candidate_mds:
        timestr = curr[1].strip()
        if len(target) != 10:
            target = '0'*(10-len(target))+target
        if len(timestr) != 10:
            timestr = '0'*(10-len(timestr))+timestr
        
        target_time = datetime.strptime(target, '%y%m%d%H%M')
        curr_time = datetime.strptime(timestr, '%y%m%d%H%M')
        diff = curr_time - target_time
        diff = abs(diff.days*24*3600+diff.seconds+diff.microseconds/1e6)
        if diff < curr_diff:
            curr_diff = diff
            return_curr = curr
    return return_curr

def divide_list(list_to_divide, ratios):
    total_elements = len(list_to_divide)
    split_index1 = int(total_elements * ratios[0])
    split_index2 = int(total_elements * ratios[1])
    
    list1 = list_to_divide[:split_index1]
    list2 = list_to_divide[split_index1:split_index2]
    list3 = list_to_divide[split_index2:]
    
    return list1, list2, list3

def find_all_files(folder, str_pattern='*.npz'):
    files = [os.path.basename(y) for x in os.walk(folder) for y in glob(os.path.join(x[0], str_pattern))]
    return files

def find_all_files_(folder, str_pattern='*.npz'):
    files = []
    for x in os.walk(folder):
        for y in glob(os.path.join(x[0], str_pattern)):
            files.append(os.path.basename(y))
        break
#     files = [os.path.basename(y) for x in os.walk(folder) for y in glob(os.path.join(x[0], str_pattern)) break]
    return files

def merge_dicts(input_dict):
    merge_dict = {}
    for x in input_dict:
        if not merge_dict:
            for k, v in x.items():
                merge_dict[k] = [v]
        else:
            for k, v in x.items():
                merge_dict[k].append(v)
    return merge_dict

In [133]:
#--------------------
# run python match_oct_vf_glaucoma.py first to generate the matching table glaucoma_xxx.csv
#--------------------

# newscanfolder = '/shared/hdds_20T/cirrus_from_dicom/scans'
# newscanfolder_p2 = '/shared/hdds_20T/cirrus_from_dicom_part2/scans'
newscanfolder = '/shared/hdds_20T/yl535/elzelab/cirrus_ongoing/scans'
newscanfolder_p2 = '/shared/external_hdds/external20TB_1/cirrus_from_dicom/scans'

# need to mount eris data elzelab by
# sshfs yl535@erisxdl.partners.org:/data/elzelab ~/elzelab

vffile = "/shared/hdds_20T/yl535/elzelab/hfa_ongoing/hfa_ongoing_merged_24-2_subset_cleaned_all_in.csv"
newoctfile = '/shared/hdds_20T/yl535/elzelab/cirrus_ongoing/metadata_OpticDiscCube_ongoing_cleaned.csv'
# exclude_file = '/shared/ssd_16T/yl535/project/python/datasets/crosssectional/test1000_pidmaps.npy'
exclude_file = ''

# save_folder = '/shared/ssd_16T/yl535/project/python/datasets/havo_crosssectional_new_export/crosssectional_fairness_all'

selected_patient_num = -1 # 3300
selected_patient_num_train = 2000 # 31031 #
balanced_num = -1 # 1000
ratio_of_patients_for_test = 0.3
num_attr = 3
extract_to_files = True
identity_type = 'race'

# save_folder = f'/shared/ssd_16T/yl535/project/python/datasets/harvard/glaucoma.csv'
match_table_file = f'/shared/ssd_16T/yl535/project/python/datasets/harvard/glaucoma_June23.csv'
save_to_folder = os.path.dirname(match_table_file)
save_file_name = os.path.basename(match_table_file)

seed = -1 # 9648 9220 for iccv Galucoma Fairness dataset # -1

if seed < 0:
    seed = int(np.random.randint(10000, size=1)[0])
set_random_seed(seed)

output_folder = f'/shared/ssd_16T/yl535/project/python/datasets/harvard/glaucoma_lancet_journal_{seed}_{selected_patient_num}'


# samples_per_attr = [int(selected_patient_num/num_attr), int(selected_patient_num/num_attr), selected_patient_num-2*int(selected_patient_num/num_attr)]
# samples_per_attr_test = [int(selected_patient_num_test/num_attr), int(selected_patient_num_test/num_attr), selected_patient_num_test-2*int(selected_patient_num_test/num_attr)]

# dict_race = {1:'American Indian or Alaska Native', 
#                 2:'Asian', 
#                 3:'Black or African American', 
#                 4:'Hispanic or Latino', 
#                 5:'Native Hawaiian or Other Pacific Islander', 
#                 6:'Other', 
#                 7:'White or Caucasian'}
# dict_race = {2:'Asian', 
#                 3:'Black or African American', 
#                 7:'White or Caucasian'}
dict_race = {2: 0, 
                3: 1, 
                7: 2}

log_output = os.path.join(save_to_folder, f'{save_file_name}_selected_{seed}.log')
log_output = None

print(f'====> seed: {seed}')

isExist = os.path.exists(save_to_folder)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(save_to_folder)

if extract_to_files:
    isExist = os.path.exists(output_folder)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(output_folder)

====> seed: 3755


In [134]:
octmetanew = pd.read_csv(match_table_file, sep=',')
# existed_paths = []
# for octpath in octmetanew['datadir']:
#     if os.path.exists(os.path.join(newscanfolder, octpath)):
#         existed_paths.append(octpath)
# octmetanew_exist = octmetanew[octmetanew['datadir'].isin(existed_paths)]

print(f'now process oct samples')
clockhours_all = []
glaucoma_all = []
datadir_all = []
octpath_all = []
md_all = []
count = 0
unmatched = []
race_all = []
male_all = []
hispanic_all = []
ilm_maps = {}
rnflt_maps = {}
rnflt_attrs = {}
upids = []
pid_all = []
timeoffset_all = []
dict_pid_data = {}
meta_all = []
for index, row in octmetanew.iterrows():
    if index % 1000 == 0:
        print(f'processing {index}')
    pid = str(row[0])
    male = int(row['male'])
    signalstrength = float(row['signalstrength'])
    righteye = str(row['righteye'])
    timeoftest = str(row['timeoftest']).strip()
    datadir = row['datadir']
    race = row['race']
    # pprint(row, log_output)
    hisp = row['hispanic']
    race = dict_race[int(race)]
    hisp = int(hisp)
    md = float(row['md'])
    glaucoma = int(row['glaucoma'])
    datetime_timeoftest = convert_to_datetime(timeoftest)
    age = float(row['age'])
    
    # if (row['race']) != 'NA' and (not np.isnan(row['race'])):
    #     race = int(row['race'])
    
    #cid = pid + '_' + righteye + '_' + male 
    cid = pid + '_' + righteye
    octID = pid + '_' + righteye + '_' + timeoftest
    
    # if cid not in vfid_timetd.keys():
    #     unmatched.append(pid)
    #     continue
    
#             fundus = cv2.imread(os.path.join(octpath, datadir, 'slo.jp2'))
    if not os.path.join(os.path.join(octpath, datadir, 'slo.jp2')):
        continue

    try:

        clockhours = []
        for i in range(1,13):
            clock_v = float(row[f'clockhour{i}'])
            clockhours.append(clock_v)
        clockhours = np.array(clockhours).astype(float)

        pid_all.append(pid)
#         race_all.append(race)
#         male_all.append(male)
#         hispanic_all.append(hisp)
#         glaucoma_all.append(glaucoma)
#         md_all.append(md)
#         datadir_all.append(datadir)
#         timeoffset_all.append(datetime_timeoftest)
#         clockhours_all.append(clockhours)
        
       
        octpath = newscanfolder
        if not os.path.exists(os.path.join(octpath, datadir)): 
            octpath = newscanfolder_p2
        octpath_all.append(octpath)
        
        data_wrt_a_sample = [pid, race, male, hisp, glaucoma, md, datadir, octpath, datetime_timeoftest, clockhours, age, row]
        if pid not in dict_pid_data:
            dict_pid_data[pid] = [data_wrt_a_sample]
        else:
            dict_pid_data[pid].append(data_wrt_a_sample)
            
        # if extract_to_files:
        #     # disc
        #     data_disk = open(os.path.join(octpath, datadir, 'mask_disc.csv'))
        #     img_disk = np.reshape([float(row) for row in data_disk], (200, -1))
        #     # cup
        #     data_cup = open(os.path.join(octpath, datadir, 'mask_cup.csv'))
        #     img_cup = np.reshape([float(row) for row in data_cup], (200, -1))
        #     # ilm
        #     data_ilm2nd = open(os.path.join(octpath, datadir, 'segmentation_ilm_2nd.csv'))
        #     img_ilm2nd = np.reshape([float(row) for row in data_ilm2nd], (200, -1))
        #     data_ilm = open(os.path.join(octpath, datadir, 'segmentation_ilm.csv'))
        #     img_ilm = np.reshape([float(row) for row in data_ilm], (200, -1))
        #     # gcl
        #     data_gcl = open(os.path.join(octpath, datadir, 'segmentation_rnfl_to_gcl.csv'))
        #     img_gcl = np.reshape([float(row) for row in data_gcl], (200, -1))
        #     # rnflt map
        #     rnflt_map = (img_gcl - img_ilm2nd) * 0.00195503 * 1000 # bscanPixelspacingDepth * 1000
        #     rnflt_map[img_disk==1] = -1
        #     rnflt_map[img_cup==1] = -2
        #     img_gcl = img_gcl * 0.00195503 * 1000
        #     img_ilm = np.abs(img_ilm-np.max(img_ilm)) * 0.00195503 * 1000

        #     bscans = []
        #     for i in range(1, 201):
        #         bscan = cv2.imread(os.path.join(octpath, datadir, f'bscan{i}.jp2'))
        #         bscan = bscan[:,:,0]
        #         bscan = cv2.rotate(bscan, cv2.ROTATE_90_CLOCKWISE)
        #         bscan = cv2.resize(bscan, (200,200))
        #         # bscan = cv2.resize(cv2.rotate(bscan, cv2.ROTATE_90_CLOCKWISE), (300,200))
        #         bscan = np.flip(bscan, axis=1)
        #         bscan = bscan[None,:,:]
        #         bscans.append(bscan)
        #     bscans = np.vstack(bscans)

        #     np.savez(os.path.join(output_folder,
        #             f"data_{index_shuf[i]:06d}.npz"), 
        #             rnflt=rnflt_map,
        #             ilm=img_ilm,
        #             oct_bscans=bscans,
        #             clockhours=clockhours,
        #             md=md,
        #             race=race,
        #             male=male,
        #             hispanic=hisp,
        #             glaucoma=glaucoma)
        #             # tds=tds,
        #             # clockhours=clockhours,
        #             # pid=octID,
        #             # datadir=datadir
        
    except Exception as e:
        unmatched.append(pid)
        print(e)

    if index % 50000 == 0:
        print(index)

unique_pids = np.unique(pid_all)
print(f'total # of patients: {len(unique_pids)}, total # of match records: {len(pid_all)}')

now process oct samples
processing 0
0
processing 1000
processing 2000
processing 3000
processing 4000
processing 5000
processing 6000
processing 7000
processing 8000
processing 9000
processing 10000
processing 11000
processing 12000
processing 13000
processing 14000
processing 15000
processing 16000
processing 17000
processing 18000
processing 19000
processing 20000
processing 21000
processing 22000
processing 23000
processing 24000
processing 25000
processing 26000
processing 27000
processing 28000
processing 29000
processing 30000
processing 31000
processing 32000
processing 33000
processing 34000
processing 35000
processing 36000
processing 37000
processing 38000
processing 39000
processing 40000
processing 41000
processing 42000
processing 43000
processing 44000
processing 45000
total # of patients: 14860, total # of match records: 45207


In [135]:
# sort samples by datetime
selected_samples = []
pid_all = []
for key, samples in dict_pid_data.items():
    if len(samples) > 1:
        datetime_wrt_a_patient = []
        for x in samples:
            datetime_wrt_a_patient.append(x[8])
        datetime_wrt_a_patient.sort()
        selected_samples.append(samples[-1])
    elif len(samples) == 1:
        selected_samples.append(samples[0])
    pid_all.append(key)
print(f'number of selected samples: {len(selected_samples)}')

number of selected samples: 14860


In [131]:
selected_patient_num = 3300

seed = -1 # 9648 9220 for iccv Galucoma Fairness dataset # -1

if seed < 0:
    seed = int(np.random.randint(10000, size=1)[0])
set_random_seed(seed)

output_folder = f'/shared/ssd_16T/yl535/project/python/datasets/harvard/glaucoma_lancet_journal_{seed}_{selected_patient_num}'

log_output = os.path.join(save_to_folder, f'{save_file_name}_selected_{seed}.log')
log_output = None

print(f'====> seed: {seed}')

if extract_to_files:
    isExist = os.path.exists(output_folder)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(output_folder)

====> seed: 3502


In [136]:
race_all_new = []
male_all_new = []
hispanic_all_new = []
age_all_new = []
clockhours_all_new = []
glaucoma_all_new = []
pid_all_new = []
meta_all = []
index_shuf = list(range(len(selected_samples)))
random.shuffle(index_shuf)
count = 0
for i in range(len(index_shuf)):
    if selected_patient_num > 0 and count >= selected_patient_num:
        break
    
    if i % 1000 == 0:
        print(f'processing {i}')

#     race = race_all[index_shuf[i]]
#     male = male_all[index_shuf[i]]
#     hisp = hispanic_all[index_shuf[i]]
#     glaucoma = glaucoma_all[index_shuf[i]]
#     clockhours = clockhours_all[index_shuf[i]]
#     md = md_all[index_shuf[i]]
#     datadir = datadir_all[index_shuf[i]]
#     octpath = octpath_all[index_shuf[i]]
#     pid = pid_all[index_shuf[i]]
    
    pid, race, male, hisp, glaucoma, md, datadir, octpath, datetime_timeoftest, clockhours, age, meta_row = selected_samples[index_shuf[i]]

    if not os.path.exists(os.path.join(octpath, datadir, 'slo.jp2')):
        continue
#     if selected_patient_num > 0 and len(glaucoma_all_new) >= selected_patient_num:
#         break

#     if pid in pid_all_new:
#         continue

#     if selected_patient_num < 0 or balanced_num < 0 or np.sum(np.array(race_all_new) == race) < balanced_num:
    race_all_new.append(race)
    male_all_new.append(male)
    hispanic_all_new.append(hisp)
    clockhours_all_new.append(clockhours)
    glaucoma_all_new.append(glaucoma)
    pid_all_new.append(pid)
    age_all_new.append(age)

    if extract_to_files:
        # disc
        data_disk = open(os.path.join(octpath, datadir, 'mask_disc.csv'))
        img_disk = np.reshape([float(row) for row in data_disk], (200, -1))
        # cup
        data_cup = open(os.path.join(octpath, datadir, 'mask_cup.csv'))
        img_cup = np.reshape([float(row) for row in data_cup], (200, -1))
        # ilm
        data_ilm2nd = open(os.path.join(octpath, datadir, 'segmentation_ilm_2nd.csv'))
        img_ilm2nd = np.reshape([float(row) for row in data_ilm2nd], (200, -1))
        data_ilm = open(os.path.join(octpath, datadir, 'segmentation_ilm.csv'))
        img_ilm = np.reshape([float(row) for row in data_ilm], (200, -1))
        # gcl
        data_gcl = open(os.path.join(octpath, datadir, 'segmentation_rnfl_to_gcl.csv'))
        img_gcl = np.reshape([float(row) for row in data_gcl], (200, -1))
        # rnflt map
        rnflt_map = (img_gcl - img_ilm2nd) * 0.00195503 * 1000 # bscanPixelspacingDepth * 1000
        rnflt_map[img_disk==1] = -1
        rnflt_map[img_cup==1] = -2
        img_gcl = img_gcl * 0.00195503 * 1000
        img_ilm = np.abs(img_ilm-np.max(img_ilm)) * 0.00195503 * 1000
        
        fundus = cv2.imread(os.path.join(octpath, datadir, 'slo.jp2'))
        fundus = cv2.rotate(fundus, cv2.ROTATE_90_CLOCKWISE)
        fundus = fundus / fundus.max() * 255
        fundus = fundus.astype(np.uint8)
#         fundus = cv2.resize(fundus, (200, 200))
#         fig, ax = plt.subplots(nrows=2, ncols=1)
#         plt.subplot(1, 2, 1)
#         plt.imshow(fundus)
#         plt.subplot(1, 2, 2)
#         plt.imshow(rnflt_map)
#         sys.exit()

        bscans = []
        for j in range(1, 201):
            bscan = cv2.imread(os.path.join(octpath, datadir, f'bscan{j}.jp2'))
            bscan = bscan[:,:,0]
            bscan = cv2.rotate(bscan, cv2.ROTATE_90_CLOCKWISE)
            bscan = cv2.resize(bscan, (200,200))
            # bscan = cv2.resize(cv2.rotate(bscan, cv2.ROTATE_90_CLOCKWISE), (300,200))
            bscan = np.flip(bscan, axis=1)
            bscan = bscan[None,:,:]
            bscans.append(bscan)
        bscans = np.vstack(bscans)

#         file_name = f"data_{index_shuf[i]:06d}.npz"
        file_name = f"data_{count:06d}.npz"
        
        tmp_dict = {'file_name': file_name, 'md': md, 'glaucoma_label': glaucoma}
        tmp_dict.update(meta_row.to_dict())
        meta_all.append(tmp_dict)
        
#         np.savez(os.path.join(output_folder, f"data_{index_shuf[i]:06d}.npz"), 
        np.savez(os.path.join(output_folder, file_name), 
                rnflt=rnflt_map,
                ilm=img_ilm,
                slo_fundus=fundus,
                oct_bscans=bscans,
                clockhours=clockhours,
                md=md,
                race=race,
                male=male,
                hispanic=hisp,
                age=age,
                glaucoma=glaucoma,
                datadir=datadir)
                # tds=tds,
                # clockhours=clockhours,
                # pid=octID,
                # datadir=datadir
        count += 1

dict_meta_all = merge_dicts(meta_all)
meta_all_df = pd.DataFrame.from_dict(dict_meta_all)
meta_all_df.to_csv(os.path.join(output_folder, 'selected_samples_meta.csv'), index=False)

processing 0
processing 1000
processing 2000
processing 3000
processing 4000
processing 5000
processing 6000
processing 7000
processing 8000
processing 9000
processing 10000
processing 11000
processing 12000
processing 13000
processing 14000


In [130]:
ratios = [0.6, 0.7, 1.]  # Divide the list into training, validation, testing proportions 60%, 10%, and 30%
# ratios = [0.6364, 0.7273, 1.] 

all_samples = find_all_files(output_folder, '*.npz')

training_samples, val_samples, testing_samples =  divide_list(all_samples, ratios)
# print(f'{len(training_samples)}, {len(val_samples)}, {len(testing_samples)}')
# sys.exit()

subset_path = os.path.join(output_folder, 'train')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in training_samples:
    shutil.move(os.path.join(output_folder, x), os.path.join(subset_path, x))

subset_path = os.path.join(output_folder, 'val')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in val_samples:
    shutil.move(os.path.join(output_folder, x), os.path.join(subset_path, x))
    
subset_path = os.path.join(output_folder, 'test')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in testing_samples:
    shutil.move(os.path.join(output_folder, x), os.path.join(subset_path, x))

In [18]:
output_folder = f'/data/home/luoy/project/python/datasets/harvard/glaucoma_lancet_journal_3755_-1'

ratios = [0.6, 0.7, 1.]  # Divide the list into training, validation, testing proportions 60%, 10%, and 30%
# ratios = [0.6364, 0.7273, 1.] 

sep_seed = -1 # 9648 9220 for iccv Galucoma Fairness dataset # -1
if sep_seed < 0:
    sep_seed = int(np.random.randint(10000, size=1)[0])
set_random_seed(sep_seed)
print(f'divide into training, validation, testing sets with seed {sep_seed}')

all_samples = find_all_files_(output_folder, '*.npz')

random.shuffle(all_samples)

training_samples, val_samples, testing_samples =  divide_list(all_samples, ratios)
print(f'{len(training_samples)}, {len(val_samples)}, {len(testing_samples)}')

subset_path = os.path.join(output_folder, f'{sep_seed}', 'train')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in training_samples:
    shutil.copy(os.path.join(output_folder, x), os.path.join(subset_path, x))

subset_path = os.path.join(output_folder, f'{sep_seed}', 'val')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in val_samples:
    shutil.copy(os.path.join(output_folder, x), os.path.join(subset_path, x))
    
subset_path = os.path.join(output_folder, f'{sep_seed}', 'test')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in testing_samples:
    shutil.copy(os.path.join(output_folder, x), os.path.join(subset_path, x))
print(f'done')

divide into training, validation, testing sets with seed 5399
8862, 1477, 4432
done


In [ ]:
ratios = [2100, 300, 900]  # Divide the list into training, validation, testing proportions 60%, 10%, and 30%

all_samples = find_all_files(output_folder, '*.npz')

training_samples, val_samples, testing_samples =  divide_list(all_samples, ratios)
print(f'{len(training_samples)}, {len(val_samples)}, {len(testing_samples)}')

subset_path = os.path.join(output_folder, 'train')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in training_samples:
    shutil.move(os.path.join(output_folder, x), os.path.join(subset_path, x))

subset_path = os.path.join(output_folder, 'val')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in val_samples:
    shutil.move(os.path.join(output_folder, x), os.path.join(subset_path, x))
    
subset_path = os.path.join(output_folder, 'test')
if not os.path.exists(subset_path):
    os.makedirs(subset_path)
for x in testing_samples:
    shutil.move(os.path.join(output_folder, x), os.path.join(subset_path, x))

In [60]:
# meta_file_path = os.path.join(output_folder, 'selected_samples_meta.csv')
meta_file_path = match_table_file
dr_meta = pd.read_csv(meta_file_path, sep=',')
duplicate_check = dr_meta['id'].duplicated().any()
print(f"exist duplicate ids: {duplicate_check}")
print(f"# of right eyes: {len(dr_meta[(dr_meta['righteye']==1)])}, # of left eyes: {len(dr_meta[(dr_meta['righteye']==0)])}, total # of eyes: {len(dr_meta)}")

exist duplicate ids: True
# of right eyes: 23962, # of left eyes: 21245, total # of eyes: 45207
